In [ ]:
import pandas as pd

import altair as alt
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder 

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
import pandas as pd

import altair as alt
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder 

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
# para encontrar os outliers 
from scipy import stats
# ferramenta para preencher os valores faltantes 
from sklearn.impute import SimpleImputer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/task-01-nba-matches-winner-prediction/train_full.csv')
df.head(2)

In [ ]:
Coluna_name=[]
for i in df:
    Coluna_name.append(i.strip()) #retirando os espaços dos nomes das colunas, se houver, para não dar erro na hora de chamar
df.columns = Coluna_name #renomeando todas as colunas

df.head(2)

Colocando valores númericos para as variáveis categóricas

In [ ]:
df.info() #vendo os tipos de dados que tenho

Consigo ver quantos dados eu deveria ter e assim vejo qual tem valores faltantes

#**Estatísticas descritivas dos dados**

In [ ]:
df.describe()

In [ ]:
# total de vitórias na temporada regular de cada time 
# create scatterplot
fig = plt.scatter(df['H_Wins'], df['H_Team'])

#adiciona rótulos de plotagem
plt.xlabel('H_Wins')
plt.ylabel('H_Team')
plt.show()

Matriz de correlação dos dados

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(10, 12))
sns.heatmap(df.corr(),
            annot = True,
            fmt = '.1f',
            linewidths=.6,
            cmap='Greens')
plt.title('Correlação entre variáveis - NBA')
plt.show()

In [ ]:
df['WinOrLose'] = df['WinOrLose'].apply(lambda x : 1 if (x == 'W') else 0)
df.head()

In [ ]:
# colunas com correlação maior que 0.2
#Encontrar as com maior acurácia para retira - lás depois
var = df.corr()['WinOrLose'].abs()>=0.2
dic = var.to_dict() 

for i, v in dic.items():
    if v is True:
        print([i])

In [ ]:
#Para vermos quais variáveis são correlacionadas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df1= df[['H_W/D %','H_MOV','A_Wins','A_Loss','A_W/D %','A_SRS','A_PL','A_MOV','WinOrLose']]
upp_mat = np.triu(df1.corr())

sns.heatmap(df1 .corr(), vmin = -1, vmax = +1, annot = True, cmap = 'coolwarm', mask = upp_mat)

In [ ]:
#Verificando balanceamento do dataframe
Losses = df['WinOrLose'].value_counts()[1]
Wins = df['WinOrLose'].value_counts()[0]

print("Derrotas:", Losses)
print("Vitórias:", Wins)


In [ ]:
import matplotlib.pyplot as plt 
  
  
Losses = df['WinOrLose'].value_counts()[1]
Wins = df['WinOrLose'].value_counts()[0]

#conjunto de dados
labels = ['Losses','Wins'] 
sizes = [Losses,Wins] 

#representação, área de plotagem 
fig1, ax1 = plt.subplots()

#gráfico
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)

#para deixar o gráfico em circulo
ax1.axis('equal')

plt.show()

Trazendo todas as primeiras análises com o Pandas profiling que retorna as correlações, principais gráficos etc

In [ ]:
# profile = ProfileReport(df, title='Analise_NBA',
#                  html={'style': {'full_width': True}}, sort = None)
# profile

## **Pré - processamento**

Split Feature e Target

In [ ]:
X = df.iloc[:, :-1] #features
y = df['WinOrLose'] #target

X.head()

In [ ]:
y

In [ ]:
X.drop(['H_W/D %', 'H_MOV', 'A_Wins', 'A_Loss','A_W/D %','A_SRS', 'A_PL', 'A_MOV'], axis = 1, inplace = True)

Categorical features

In [ ]:
obj = (X.dtypes == 'object') #quais colunas que são categóricas dentro do dataset?
object_cols = list(obj[obj].index)

object_cols

In [ ]:
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(X[object_cols])

X.head(2)

Outliers

In [ ]:
out = sns.boxplot(data=df1, orient="h", palette="Set2")

In [ ]:
# # Método MAD
# # para indentificar os outliers em determinada coluna 
# def mad_method(df, variable_name):
#     # Recebe dois parâmetros: df e variável de interesse
#     columns = df.columns
#     med = np.median(df, axis = 0)
#     mad = np.abs(stats.median_absolute_deviation(df))
#     threshold = 3
#     outlier = []
#     index=0
#     for item in range(len(columns)):
#         if columns[item] == variable_name:
#             index == item
#     for i, v in enumerate(df.loc[:,variable_name]):
#         t = (v-med[index])/mad[index]
#         if t > threshold:
#             outlier.append(i)
#         else:
#             continue
#     return outlier

# outlier_mad = mad_method(df, 'A_PL')
# print(outlier_mad)

Tratando valores faltantes


In [ ]:
# ferramenta do SKlearn que faz o preenchimento dos valores faltantes 
# instancia o objeto
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

x = df

imp_mean.fit(X)

print(imp_mean.transform(X))

Split

In [ ]:
from sklearn.model_selection import train_test_split

trainX, valX, trainy, valy = train_test_split(X,
                                                          y,
                                                          test_size=0.3, #70 treino / 30 teste
                                                          random_state=12,
                                                          )
valX

Normalização


In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range=(0, 1))

trainX = sc.fit_transform(trainX)

valX = sc.transform(valX)

In [ ]:
X

Modelo

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(max_depth=6, n_estimators=300) #modelo
model.fit(trainX, trainy)
predict =  model.predict(valX)

predict

In [ ]:
print(cross_val_score(model,X , y, cv=5, scoring='balanced_accuracy').mean()); #cross validation está aq

In [ ]:
# KNN

from sklearn.neighbors import KNeighborsClassifier

model2 = KNeighborsClassifier(n_neighbors = 3)
model2.fit(X, y)

predict2 =  model.predict(valX)

predict2

In [ ]:
# avaliando o desempenho do KNN

from sklearn.model_selection import cross_val_score

scores_dt = cross_val_score(model, X, y, scoring='accuracy', cv=5)
print(scores_dt.mean())

Predição Final

In [ ]:
df_test = pd.read_csv('/kaggle/input/task-01-nba-matches-winner-prediction/test_without_label.csv')
df_test.head(1)

Tirar os espaços em branco das colunas

In [ ]:
Coluna_name=[]
for i in df_test:
    Coluna_name.append(i.strip()) #retirando os espaços dos nomes das colunas, se houver, para não dar erro na hora de chamar
df_test.columns = Coluna_name #renomeando todas as colunas

df_test.head(2)

In [ ]:
df_test.drop(['H_W/D %', 'H_MOV', 'A_Wins', 'A_Loss','A_W/D %','A_SRS', 'A_PL', 'A_MOV'], axis = 1, inplace = True)

In [ ]:
obj = (df_test.dtypes == 'object') #quais colunas que são categóricas dentro do dataset?
object_cols = list(obj[obj].index)

object_cols

In [ ]:
ordinal_encoder = OrdinalEncoder()
df_test[object_cols] = ordinal_encoder.fit_transform(df_test[object_cols])

df_test.head(2)

In [ ]:
df_test['winorlose'] = pd.Series(model.predict(df_test))

In [ ]:
df_test['winorlose'] = df_test['winorlose'].map({ 0 : 'L',
                                                  1 : 'W'})
df_test

In [ ]:
df_test[['Game','winorlose']].to_csv('/kaggle/working/submission.csv', index = False)